# W&B Sweep Launcher
### $Time$ $Series$ $3rd$ $Test$

$Vasco$ $Mergulhão$ $-$ $March$ $2023$

### Version 1:
 - Applies Weights and Biases Sweeps on a given Sub-Sample.
 - Imports Custom Functions and Networks


### ANN Configurations:

- #### Architecture(s)
    - Fully Connected Auto Encoder
        - Small (Input, 200, 200, LatDim)
        - Medium (Input, 300, 300, 300, 300, LatDim)
        - N2D [other papers orig ref] (Input, 500, 500, 2000, LatDim)
    
- #### Hyperparamenters
    - Latent Space Size
    - Batch Size
        - Small test [2 - 32] and Large test [128 - 256]
    - Learning Rate
    - Learning Rate Scheduler
        - Performance Schedulling
    - Activation Functions
        - SELU and Leaky ReLU
    - Initializations
        - LeCun and He (accordingly)
    - Batch Normalization
        - With/Without tests (note: if data is not z-scored, SELU not worth it, downgrade to ELU)
    - Optimizers
        - Nadam and SDG(momentum [0.9], Nesterov)
    - Epochs
        - 100 with Early Stopping
 

---
# 

In [1]:
# Custom Functions
from Data_Processing import Transform, Clean
from networks import ann_train, fc_small, fc_medium, fc_n2d

In [2]:
# Standard Libraries
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from sklearn.model_selection import train_test_split
import wandb
from wandb.keras import WandbCallback

---
# Script Variables

In [3]:
# Defines Dataset for the Sweep
dataset_name = 'Kenya_10k_Set_1_w90'

# Uses name to navigate folders
dataset_folder = "_".join(dataset_name.split('_')[:-1]) #Takes out window length section
dataset_location = f'../Data/{dataset_folder}/{dataset_name}.csv'

# Zcore Data Decision
zscore_data = True # Set to: [True/False]
zscore_data_done = False # Always set to False. Ensures its not normalized multiple times

# Model Name and Variables
AE_Model_Name = 'FC_N2D' # Options: FC_Small, FC_Medium, FC_N2D
latent_layer_size = 25

# Sweep Names and Configurations
Project_Name = f'FC_AE-{dataset_name}'
Sweep_Config = f'{AE_Model_Name}_sweepconfig'
sweep_count = 1

In [4]:
def window_col_names(dataset_name, win_prefix = 'd'):
    # retriving window length
    window_len = int(dataset_name.split('_')[-1][1:]) # Gets _wXX part of name, then skips 'w' to get the number.
    # defining window column names
    window_cols = [None]*window_len
    for i  in range(window_len):
        window_cols[i] = f'{win_prefix}' + str(i+1)
        
    return window_cols, window_len

window_cols, window_len = window_col_names(dataset_name)

In [5]:
if AE_Model_Name == 'FC_N2D':
    sweep_config = fc_n2d.sweep_config(name=Sweep_Config, window_len=window_len, latent_layer_size=latent_layer_size)
    ann_network = fc_n2d.model(window_length = window_len, latent_layer_size = latent_layer_size, activation_fn = 'SELU')
    
elif AE_Model_Name == 'FC_Medium':
    sweep_config = fc_medium.sweep_config(name=Sweep_Config, window_len=window_len, latent_layer_size=latent_layer_size)
    ann_network = fc_medium.model(window_length = window_len, latent_layer_size = latent_layer_size, activation_fn = 'SELU')
    
elif AE_Model_Name == 'FC_Small':
    sweep_config = fc_small.sweep_config(name=Sweep_Config, window_len=window_len, latent_layer_size=latent_layer_size)
    ann_network = fc_small.model(window_length = window_len, latent_layer_size = latent_layer_size, activation_fn = 'SELU')
    
else:
    print(f'ERROR: AE name {AE_Model_Name} not recognised!')

In [6]:
sweep_config

{'method': 'random',
 'name': 'FC_N2D_sweepconfig',
 'metric': {'name': 'mse', 'goal': 'minimize'},
 'parameters': {'optimizer': {'values': ['nadam', 'sgd']},
  'latent_layer_size': {'value': 25},
  'epochs': {'value': 100},
  'window_length': {'value': 90},
  'activation_fn': {'values': ['SELU', 'LeakyReLU']},
  'learning_rate': {'distribution': 'log_uniform_values',
   'min': 1e-05,
   'max': 0.01},
  'batch_size': {'distribution': 'q_log_uniform_values',
   'q': 2,
   'min': 20,
   'max': 256}}}

In [7]:
ann_network.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 90)]              0         
                                                                 
 dense (Dense)               (None, 500)               45500     
                                                                 
 dense_1 (Dense)             (None, 500)               250500    
                                                                 
 dense_2 (Dense)             (None, 2000)              1002000   
                                                                 
 dense_3 (Dense)             (None, 25)                50025     
                                                                 
 dense_4 (Dense)             (None, 2000)              52000     
                                                                 
 dense_5 (Dense)             (None, 500)               100050

---
# Data Imports

In [8]:
Data = pd.read_csv(dataset_location)

In [9]:
Data.head()

,short_ID,window_ID,window_start_date,d1,d2,d3,d4,d5,d6,d7,...,d81,d82,d83,d84,d85,d86,d87,d88,d89,d90
0,347,0,2018-01-19,15.422222,14.422222,13.422222,12.422222,11.422222,10.422222,9.422222,...,8.696088,6.709653,6.696088,5.696088,3.730243,3.696088,1.740706,0.744016,-7.000000,-7.000000
1,347,1,2018-04-19,6.475880,4.490544,4.475880,3.475880,2.475880,1.475880,-7.000000,...,-7.000000,-7.000000,-7.000000,-7.000000,-7.000000,-7.000000,-7.000000,6.634468,5.634468,4.634468
2,347,2,2018-07-18,3.634468,2.634468,1.634468,3.634468,2.634468,1.634468,0.634468,...,4.762303,3.762303,2.762303,1.762303,0.762303,5.762303,4.762303,3.762303,2.762303,1.762303
3,347,3,2018-10-16,0.762303,1.762303,0.762303,2.762303,1.762303,1.762303,0.762303,...,3.553345,2.553345,1.553345,0.553345,-7.000000,6.646019,5.646019,4.646019,3.646019,2.646019
4,347,4,2019-01-14,1.646019,0.646019,2.646019,1.646019,0.646019,2.646019,1.646019,...,0.613681,2.613681,1.613681,1.613681,0.613681,1.613681,2.613681,1.613681,1.613681,0.613681


---
# Pre-Processing

---
## Z-Scoring Data

This is done on a row-by-row basis.<br>
Meaning, each window is normalized to its own Mean and Std.

In [10]:
if zscore_data == True and zscore_data_done == False:
    Data = Transform.Zscore(Data, window_cols)
    zscore_data_done = True
    print('Data WAS Zscored')
    
elif zscore_data == True and zscore_data_done == True:
    print('Already WAS Zscored')
    
elif zscore_data == False:
    print('Data NOT Zscored')
    
else:
    print('Error')

Data WAS Zscored


---
## NaN Checks and Policy

In [11]:
Data = Clean.NaN_policy(Data, window_cols)

NaN values detected.
There are 22 rows of all NaN values.
Rows of NaN corrected
NaNs no longer detected.


---
## Train-Test Split

In [12]:
data_array = Data[window_cols].to_numpy()

In [13]:
x_train, x_test = train_test_split(data_array, test_size=0.2, random_state=42)

---
---
# WandB Sweep Log in
https://github.com/wandb/examples/blob/master/colabs/keras/Keras_param_opti_using_sweeps.ipynb


In [14]:
wandb.login()

wandb: Currently logged in as: vasco-mergulhao (vasco-phd). Use `wandb login --relogin` to force relogin


True

# Sweep & Train Functions

In [15]:
def train(model, batch_size= 32, epochs= 100, lr=0.001, optimizer='nadam'):  
    
    tf.keras.backend.clear_session()
    model.compile(loss="mse", 
                  optimizer=ann_train.get_optimizer(lr, optimizer), 
                  metrics=["mse"])

    early_stopping_cb = tf.keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True)
    lr_scheduler_cb = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)

    model.fit(x_train, 
              x_train, 
              batch_size=batch_size, 
              epochs=epochs, 
              validation_data=(x_test, x_test), 
              callbacks=[WandbCallback(), early_stopping_cb, lr_scheduler_cb])
    
    

In [16]:
def sweep_train(config_defaults=None):
    # Initialize wandb with a sample project name
    with wandb.init(config=config_defaults):  # this gets over-written in the Sweep

        # Specify the other hyperparameters to the configuration
        wandb.config.architecture_name = AE_Model_Name
        wandb.config.dataset_name = dataset_name
        
        train_go = True
        # initialize model
        if AE_Model_Name == 'FC_Small':
            AE_model = fc_small.model(window_length = wandb.config.window_length,
                                      latent_layer_size = wandb.config.latent_layer_size,
                                      activation_fn = wandb.config.activation_fn)
            
        elif AE_Model_Name == 'FC_Medium':
            AE_model = fc_medium.model(wandb.config.window_length,
                                       wandb.config.latent_layer_size,
                                       wandb.config.activation_fn)
            
        elif AE_Model_Name == 'FC_N2D':
            AE_model = fc_n2d.model(wandb.config.window_length,
                                    wandb.config.latent_layer_size,
                                    wandb.config.activation_fn)
        else:
            print('ERROR: AE name not recognised!')
            train_go = False
            
        if train_go:
            train(AE_model, 
                  wandb.config.batch_size, 
                  wandb.config.epochs,
                  wandb.config.learning_rate,
                  wandb.config.optimizer)
        


---
---
# Run Sweep

In [17]:
sweep_id = wandb.sweep(sweep_config, project = Project_Name)

Create sweep with ID: 7c3cr8jz
Sweep URL: https://wandb.ai/vasco-phd/FC_AE-Kenya_10k_Set_1_w90/sweeps/7c3cr8jz


In [18]:
wandb.agent(sweep_id, function=sweep_train, count= sweep_count)

wandb: Agent Starting Run: 6loq1dkt with config:
wandb: 	activation_fn: LeakyReLU
wandb: 	batch_size: 66
wandb: 	epochs: 100
wandb: 	latent_layer_size: 25
wandb: 	learning_rate: 0.0004290395910843444
wandb: 	optimizer: sgd
wandb: 	window_length: 90


wandb: WARNING The save_model argument by default saves the model in the HDF5 format that cannot save custom objects like subclassed models and custom layers. This behavior will be deprecated in a future release in favor of the SavedModel format. Meanwhile, the HDF5 model is saved as W&B files and the SavedModel as W&B Artifacts.


Epoch 1/100
1074/1074 [==============================] - ETA: 0s - loss: 0.7140 - mse: 0.7140

INFO:tensorflow:Assets written to: C:\Users\ucesvpm\OneDrive - University College London\PhD Project\Data Analytics\Time Series Clustering\Third Test\Time-Series-PhD\wandb\run-20230302_163622-6loq1dkt\files\model-best\assets


INFO:tensorflow:Assets written to: C:\Users\ucesvpm\OneDrive - University College London\PhD Project\Data Analytics\Time Series Clustering\Third Test\Time-Series-PhD\wandb\run-20230302_163622-6loq1dkt\files\model-best\assets
wandb: Adding directory to artifact (C:\Users\ucesvpm\OneDrive - University College London\PhD Project\Data Analytics\Time Series Clustering\Third Test\Time-Series-PhD\wandb\run-20230302_163622-6loq1dkt\files\model-best)... Done. 0.1s


...

INFO:tensorflow:Assets written to: C:\Users\ucesvpm\OneDrive - University College London\PhD Project\Data Analytics\Time Series Clustering\Third Test\Time-Series-PhD\wandb\run-20230302_163622-6loq1dkt\files\model-best\assets


INFO:tensorflow:Assets written to: C:\Users\ucesvpm\OneDrive - University College London\PhD Project\Data Analytics\Time Series Clustering\Third Test\Time-Series-PhD\wandb\run-20230302_163622-6loq1dkt\files\model-best\assets
wandb: Adding directory to artifact (C:\Users\ucesvpm\OneDrive - University College London\PhD Project\Data Analytics\Time Series Clustering\Third Test\Time-Series-PhD\wandb\run-20230302_163622-6loq1dkt\files\model-best)... Done. 0.1s


1074/1074 [==============================] - 19s 18ms/step - loss: 0.2670 - mse: 0.2670 - val_loss: 0.2721 - val_mse: 0.2721 - lr: 2.1452e-04
Epoch 89/100
1074/1074 [==============================] - 16s 15ms/step - loss: 0.2669 - mse: 0.2669 - val_loss: 0.2723 - val_mse: 0.2723 - lr: 2.1452e-04
Epoch 90/100
 105/1074 [=>............................] - ETA: 18s - loss: 0.2707 - mse: 0.2707

epoch,▁▁▁▁▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇███
loss,█▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
mse,█▃▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val_mse,█▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
best_epoch,87
best_val_loss,0.27209
epoch,88
loss,0.26686
mse,0.26686
val_loss,0.27229


wandb: Ctrl + C detected. Stopping sweep.


In [19]:
wandb.finish()